# TP 2 — Démarrer un projet de zéro : importer, visualiser, modéliser

**Auteur : TUELEAU Tom**  
**Date : 5 septembre 2025**

**Objectifs :**
- Créer l’ossature d’un projet (`venv`, arborescence).
- Importer un jeu de données (URL publique, fichier local, jeu intégré).
- Explorer et **visualiser** les données.
- Entraîner un **premier** modèle de machine learning (intro).

In [ ]:
# (Optionnel) Créer une arborescence minimale pour le projet
# Vous pouvez exécuter cette cellule une fois au début.
import os
for d in ["data", "notebooks", "src"]:
    os.makedirs(d, exist_ok=True)
print("Dossiers prêts :", os.listdir("."))

In [ ]:
# Dépendances (exécuter si besoin) :
# %pip install pandas matplotlib scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.datasets import load_wine

# Paramètres graphiques (taille par défaut)
plt.rcParams["figure.figsize"] = (6, 4)

## 1) Choisir et charger un dataset

- `DATASET = "penguins"` (via URL publique ; alternative locale `data/penguins.csv`).
- `DATASET = "wine"` (jeu intégré scikit‑learn, pas d’Internet requis).

In [ ]:
# Choix du dataset : "penguins" ou "wine"
DATASET = "penguins"  # <-- changez ici si vous voulez utiliser "wine"

df = None
target_col = None
target_names = None

if DATASET == "penguins":
    URL = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
    try:
        df = pd.read_csv(URL)
    except Exception as e:
        print("Échec du chargement par URL, tentative locale 'data/penguins.csv'...", e)
        df = pd.read_csv("data/penguins.csv")  # Assurez-vous d'avoir ce fichier si vous êtes hors-ligne
    target_col = "species"
    target_names = sorted(df[target_col].dropna().unique().tolist())
elif DATASET == "wine":
    wine = load_wine(as_frame=True)
    df = wine.frame.copy()     # features + target (0,1,2)
    target_col = "target"
    target_names = list(wine.target_names)
else:
    raise ValueError("DATASET doit être 'penguins' ou 'wine'.")

print("Dataset :", DATASET, "| Forme :", df.shape)
df.head()

In [ ]:
## Inspection rapide
display(df.head())
print("\n=== Info ===")
df.info()
print("\n=== Statistiques (numériques) ===")
display(df.select_dtypes(include="number").describe())
print("\n=== Valeurs manquantes par colonne ===")
display(df.isna().sum())

In [ ]:
## 2) Préparation : sélection colonnes numériques + cible

# Déterminer toutes les colonnes numériques
all_num = df.select_dtypes(include="number").columns.tolist()

# Pour 'penguins', on retient des colonnes num. courantes si présentes
penguins_default = ["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"]
if DATASET == "penguins":
    num_cols = [c for c in penguins_default if c in df.columns]
else:
    # 'wine' : on peut prendre les 6 premières colonnes numériques pour simplifier les graphiques
    num_cols = [c for c in all_num if c != target_col][:6]

# Nettoyage minimal : garder lignes complètes sur les colonnes choisies + la cible
keep_cols = num_cols + [target_col]
df_clean = df[keep_cols].dropna().copy()

# Cible catégorielle lisible
if DATASET == "penguins":
    df_clean[target_col] = df_clean[target_col].astype("category")
    class_labels = list(df_clean[target_col].cat.categories)
else:
    # 'wine' : classes 0,1,2 -> noms explicites si dispo
    df_clean[target_col] = df_clean[target_col].astype("category")
    class_labels = target_names  # ['class_0','class_1','class_2'] dans wine.target_names

X = df_clean[num_cols]
y = df_clean[target_col].astype("category")

print("Variables numériques utilisées :", num_cols)
print("Forme X/y :", X.shape, y.shape)
print("Classes :", list(y.cat.categories))

In [ ]:
## 3) Visualisations — distributions (histogrammes)
for col in num_cols:
    plt.figure()
    X[col].hist(bins=20)
    plt.title(f"Distribution de {col}")
    plt.xlabel(col); plt.ylabel("Fréquence")
    plt.tight_layout(); plt.show()

In [ ]:
## 3) Visualisations — variabilité par classe (boîtes à moustaches)
for col in num_cols:
    plt.figure()
    data_by_class = [X.loc[y==cl, col] for cl in y.cat.categories]
    plt.boxplot(data_by_class, labels=list(y.cat.categories))
    plt.title(f"{col} par classe")
    plt.xlabel("Classe"); plt.ylabel(col)
    plt.tight_layout(); plt.show()

In [ ]:
## 3) Visualisations — relations entre variables (nuages de points)
# Construire jusqu'à deux paires (a,b) pour ne pas surcharger
pairs = []
for i in range(0, min(len(num_cols), 4), 2):
    if i+1 < len(num_cols):
        pairs.append((num_cols[i], num_cols[i+1]))

codes = y.cat.codes  # 0..K-1 (utilisé pour colorer sans choisir de couleurs spécifiques)
for a,b in pairs:
    plt.figure()
    plt.scatter(X[a], X[b], c=codes)
    plt.title(f"{a} vs {b} (couleur = classe)")
    plt.xlabel(a); plt.ylabel(b)
    cbar = plt.colorbar(ticks=sorted(codes.unique()))
    try:
        cbar.ax.set_yticklabels(list(y.cat.categories))
    except Exception:
        pass
    plt.tight_layout(); plt.show()

In [ ]:
## 3) Visualisations — corrélations (matrice)
corr = X.corr(numeric_only=True)
fig, ax = plt.subplots()
cax = ax.matshow(corr.values)
fig.colorbar(cax)
ax.set_xticks(range(len(corr.columns)))
ax.set_yticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=45, ha="left")
ax.set_yticklabels(corr.columns)
plt.title("Corrélations entre variables")
plt.tight_layout(); plt.show()

## 4) Introduction au machine learning (classification)
Workflow minimal : séparation train/test → standardisation → apprentissage → évaluation.

In [ ]:
# Découpage + pipeline standardisation + régression logistique
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

print("Accuracy :", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, pred, labels=list(y.cat.categories))
fig, ax = plt.subplots()
im = ax.imshow(cm)
ax.set_xticks(range(len(y.cat.categories))); ax.set_yticks(range(len(y.cat.categories)))
ax.set_xticklabels(y.cat.categories, rotation=45, ha="right")
ax.set_yticklabels(y.cat.categories)
plt.title("Matrice de confusion")
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j], ha="center", va="center")
plt.tight_layout(); plt.show()

## 5) À rendre / Questions guidées
1. **Import** : testez `penguins` puis `wine`. Avantages/inconvénients (qualité des colonnes, propreté, taille) ?  
2. **Nettoyage** : listez les colonnes avec NA et justifiez votre stratégie (suppression vs imputation).  
3. **Visualisation** : pour chaque graphique, écrivez une phrase d’interprétation (tendance, dispersion, séparabilité des classes).  
4. **Modèle** : remplacez la régression logistique par `KNeighborsClassifier` puis `DecisionTreeClassifier`. Comparez l’accuracy.  
5. **Reproductibilité** : exportez l’environnement (`pip freeze > requirements.txt`) et donnez les étapes pour relancer le projet sur une autre machine.